In [1]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import shapely
from shapely import Point, box
import xagg as xa
import rioxarray

# supress warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

Found esmf.mk at: /home/malasawedah/.conda/envs/test2/lib/esmf.mk


#### Test

In [2]:
import pytest
import pandas as pd
import numpy as np
import xarray as xr
import shutil
import geopandas as gpd
from geopandas import testing as gpdt
from unittest import TestCase
from shapely.geometry import Polygon
from functools import reduce

import pytest
import pandas as pd
import numpy as np
import xarray as xr
import shutil
import geopandas as gpd
from geopandas import testing as gpdt
from unittest import TestCase
from shapely.geometry import Polygon

from xagg.core import (process_weights,create_raster_polygons,get_pixel_overlaps,aggregate,read_wm)
from xagg.wrappers import (pixel_overlaps)


### test_export

In [4]:
ds = xr.Dataset({'test':(['lon','lat','run'],np.array([[[0,1],[2,3]],[[0,1],[2,3]]])),
				 'lat_bnds':(['lat','bnds'],np.array([[-0.5,0.5],[0.5,1.5]])),
				 'lon_bnds':(['lon','bnds'],np.array([[-0.5,0.5],[0.5,1.5]]))},
				coords={'lat':(['lat'],np.array([0,1])),
						'lon':(['lon'],np.array([0,1])),
						'run':(['run'],np.array([0,1])),
						'bnds':(['bnds'],np.array([0,1]))})

In [5]:
# Create polygon covering multiple pixels
gdf = {'name':['test'],
			'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")

# Get pixel overlaps
wm = pixel_overlaps(ds,gdf)

# Get aggregate
agg = aggregate(ds,wm, stat=['weighted_mean'])

creating polygons for each pixel...
calculating overlaps between pixels and output polygons...
success!
aggregating test by weighted_mean...
all variables aggregated to polygons!


In [6]:
################# Test to data set ################
ds_out = agg.to_dataset()

# Build reference output dataset
ds_ref = xr.Dataset({'name':(['poly_idx'],np.array(['test']).astype(object)),
                     'test_weighted_mean':(['poly_idx','run'],np.array([[1.0,2.0]]))},
                coords={'poly_idx':(['poly_idx'],np.array([0])),
                        'run':(['run'],np.array([0,1]))})

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
xr.testing.assert_allclose(ds_out,ds_ref,atol=0.0001)

In [7]:
################# Test to dataframe ################


df_out = agg.to_dataframe()

# Build reference output dataframe
df_ref = pd.DataFrame({'poly_idx':[0,0],'run':[0,1],'name':['test','test'],'test_weighted_mean':[0.9999,1.9999]})
df_ref = df_ref.set_index(['poly_idx','run'])

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
pd.testing.assert_frame_equal(df_out,df_ref,atol=0.0001)

### test_core

In [8]:
# build raster polygons from a simple 2x2 grid of lat/lon pixels
ds = xr.Dataset({'test':(['lon','lat'],np.array([[0,1],[2,3]])),
				 'lat_bnds':(['lat','bnds'],np.array([[-0.5,0.5],[0.5,1.5]])),
				 'lon_bnds':(['lon','bnds'],np.array([[-0.5,0.5],[0.5,1.5]]))},
				coords={'lat':(['lat'],np.array([0,1])),
						'lon':(['lon'],np.array([0,1])),
						'bnds':(['bnds'],np.array([0,1]))})

# add a simple weights grid
weights = xr.DataArray(data=np.array([[0,1],[2,3]]),
							dims=['lat','lon'],
							coords=[ds.lat,ds.lon])

# calculate the pix_agg variable tested above, to be used in the 
# tests below
pix_agg = create_raster_polygons(ds,weights=weights)

In [9]:
############# test_get_pixel_overlaps_gdf_wpreexisting_index  ##################
ds1 = xr.Dataset({'test':(['lon','lat'],np.array([[np.nan,np.nan],[np.nan,np.nan]])),
                 'lat_bnds':(['lat','bnds'],np.array([[-0.5,0.5],[0.5,1.5]])),
                 'lon_bnds':(['lon','bnds'],np.array([[-0.5,0.5],[0.5,1.5]]))},
                coords={'lat':(['lat'],np.array([0,1])),
                        'lon':(['lon'],np.array([0,1])),
                        'bnds':(['bnds'],np.array([0,1]))})

# get aggregation mapping
pix_agg = create_raster_polygons(ds1)

# Create polygon covering multiple pixels
gdf = {'name':['test'],
            'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")


# Get pixel overlaps
wm = get_pixel_overlaps(gdf,pix_agg)

# Get aggregate
agg = aggregate(ds,wm)

aggregating test by weighted_mean...
all variables aggregated to polygons!


In [10]:
############# test_aggregate_basic #################
gdf = {'name':['test'],
            'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")

# calculate the pix_agg variable tested above, to be used in the 
# tests below
pix_agg = create_raster_polygons(ds)

# Get pixel overlaps
wm = get_pixel_overlaps(gdf,pix_agg)

# Get aggregate
agg = aggregate(ds,wm)

# This requires shifting rtol to 1e-4 for some reason, in that 
# it's actually 1.499981, whereas multiplying out 
# np.sum(agg.agg.rel_area[0]*np.array([0,1,2,3]))gives 1.499963... 
# Possibly worth examining more closely later
assert np.allclose([v for v in agg.agg.test_weighted_mean.values],1.4999,rtol=1e-4)

aggregating test by weighted_mean...
all variables aggregated to polygons!


In [11]:
################# test_aggregate_with_weights  #################

gdf = {'name':['test'],
            'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")

# add a simple weights grid (equator pixels have weight 1, 
# 1 N pixels have weight 0)
weights = xr.DataArray(data=np.array([[1,1],[0,0]]),
                            dims=['lat','lon'],
                            coords=[ds.lat,ds.lon])

# calculate the pix_agg variable tested above, to be used in the 
# tests below
pix_agg = create_raster_polygons(ds,weights=weights)


# Get pixel overlaps
wm = get_pixel_overlaps(gdf,pix_agg)

# Get aggregate
agg = aggregate(ds,wm)

# Since the "test" for the input ds has [0,2] for the two 
# equatorial pixels, the average should just be 1.0
assert np.allclose([v for v in agg.agg.test_weighted_mean.values],1.0)

aggregating test by weighted_mean...
all variables aggregated to polygons!


In [12]:
################ test_aggregate_with_mismatched_grid  ###############
ds = xr.Dataset({'test':(['lon','lat'],np.array([[30,40,50],[10,0,1],[20,2,3]])),
             'lat_bnds':(['lat','bnds'],np.array([[-1.5,-0.5],[-0.5,0.5],[0.5,1.5]])),
             'lon_bnds':(['lon','bnds'],np.array([[-1.5,-0.5],[-0.5,0.5],[0.5,1.5]]))},
            coords={'lat':(['lat'],np.array([-1,0,1])),
                    'lon':(['lon'],np.array([-1,0,1])),
                    'bnds':(['bnds'],np.array([0,1]))})


# Create polygon covering multiple pixels
gdf = {'name':['test'],
            'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")

# calculate the pix_agg variable tested above, to be used in the 
# tests below
pix_agg = create_raster_polygons(ds)


# Get pixel overlaps
wm = get_pixel_overlaps(gdf,pix_agg)

# Get aggregate
agg = aggregate(ds,wm)

# On change in rtol, see note in test_aggregate_basic
assert np.allclose([v for v in agg.agg.test_weighted_mean.values],1.4999,rtol=1e-4)

aggregating test by weighted_mean...
all variables aggregated to polygons!


In [13]:
################ test_aggregate_with_all_nans  ####################
ds = xr.Dataset({'test':(['lon','lat'],np.array([[np.nan,np.nan],[np.nan,np.nan]])),
                 'lat_bnds':(['lat','bnds'],np.array([[-0.5,0.5],[0.5,1.5]])),
                 'lon_bnds':(['lon','bnds'],np.array([[-0.5,0.5],[0.5,1.5]]))},
                coords={'lat':(['lat'],np.array([0,1])),
                        'lon':(['lon'],np.array([0,1])),
                        'bnds':(['bnds'],np.array([0,1]))})

# get aggregation mapping
pix_agg = create_raster_polygons(ds)

# Create polygon covering multiple pixels
gdf = {'name':['test'],
            'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")


# Get pixel overlaps
wm = get_pixel_overlaps(gdf,pix_agg)

# Get aggregate
agg = aggregate(ds,wm)

# Should only return nan 
# (this is not a great assert - but agg.agg.test[0] comes out as [array(nan)], 
# which... I'm not entirely sure how to reproduce. It quaks like a single nan,
# but it's unclear to me how to get it to work)
assert np.all([np.isnan(k) for k in agg.agg.test])

aggregating test by weighted_mean...
all variables aggregated to polygons!
